In [58]:
import numpy as np
import os
root_logdir = os.path.join(os.curdir, "my_logs")

In [59]:
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()
run_logdir

'./my_logs/run_2020_04_30-22_10_42'

In [60]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [61]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[8]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])    
model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3))

In [62]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5", save_best_only=True)


In [63]:
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint_cb, tensorboard_cb])

Train on 11610 samples, validate on 3870 samples
Epoch 1/30
11610/11610 [==============================] - 1s 99us/sample - loss: 1.8766 - val_loss: 0.8114
Epoch 2/30
11610/11610 [==============================] - 1s 47us/sample - loss: 0.6723 - val_loss: 0.6588
Epoch 3/30
11610/11610 [==============================] - 1s 47us/sample - loss: 0.6069 - val_loss: 0.6142
Epoch 4/30
11610/11610 [==============================] - 1s 48us/sample - loss: 0.5739 - val_loss: 0.5837
Epoch 5/30
11610/11610 [==============================] - 1s 68us/sample - loss: 0.5489 - val_loss: 0.5619
Epoch 6/30
11610/11610 [==============================] - 1s 51us/sample - loss: 0.5293 - val_loss: 0.5440
Epoch 7/30
11610/11610 [==============================] - 1s 74us/sample - loss: 0.5138 - val_loss: 0.5297
Epoch 8/30
11610/11610 [==============================] - 1s 54us/sample - loss: 0.5006 - val_loss: 0.5169
Epoch 9/30
11610/11610 [==============================] - 1s 65us/sample - loss: 0.4917 - val_l

In [66]:
%load_ext tensorboard.notebook
%tensorboard --logdir=./my_logs --port=6006

The tensorboard.notebook extension is already loaded. To reload it, use:
  %reload_ext tensorboard.notebook


Reusing TensorBoard on port 6006 (pid 78674), started -1 day, 23:35:50 ago. (Use '!kill 78674' to kill it.)

In [67]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [68]:
housing = fetch_california_housing()

In [69]:
housing['data'][:3].round(2)

array([[ 8.3300e+00,  4.1000e+01,  6.9800e+00,  1.0200e+00,  3.2200e+02,
         2.5600e+00,  3.7880e+01, -1.2223e+02],
       [ 8.3000e+00,  2.1000e+01,  6.2400e+00,  9.7000e-01,  2.4010e+03,
         2.1100e+00,  3.7860e+01, -1.2222e+02],
       [ 7.2600e+00,  5.2000e+01,  8.2900e+00,  1.0700e+00,  4.9600e+02,
         2.8000e+00,  3.7850e+01, -1.2224e+02]])

In [70]:
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

In [71]:
scaler = StandardScaler()

In [72]:
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [73]:
import tensorflow as tf
from tensorflow import keras
#from keras import layers
from keras.models import Model
# from keras.models import Model # this is same
from keras.layers import Dense, Input, concatenate
from keras import layers
from keras import models
# this concatenate is different from merge.Concatenate

In [74]:
X_train.shape

(11610, 8)

In [75]:
#better specify all tracks of functions in keras
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.Concatenate()([input_A, hidden2])
output = keras.layers.Dense(1, name="output")(concat)
aux_output = keras.layers.Dense(1, name='aux_output')(hidden2)
model = keras.models.Model(inputs=[input_A, input_B], outputs=[output, aux_output])

In [76]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
deep_input (InputLayer)         [(None, 6)]          0                                            
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 30)           210         deep_input[0][0]                 
__________________________________________________________________________________________________
wide_input (InputLayer)         [(None, 5)]          0                                            
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 30)           930         dense_3[0][0]                    
______________________________________________________________________________________________

In [77]:
model.compile(loss = ['mean_squared_error','mean_squared_error'], optimizer='adam', loss_weights=[0.9,0.1]) #, metrics=['accuracy','accuracy']) # adam or sgd

In [78]:
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

In [79]:
#checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model_callback.h5")
#early_stopping_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [80]:
history = model.fit((X_train_A, X_train_B), (y_train, y_train), epochs=10, 
                    validation_data=((X_valid_A, X_valid_B), (y_valid, y_valid)),callbacks=[tensorboard_cb]) 
                    #callbacks=[checkpoint_cb]) #, early_stopping_cb])
# set up checkpoints for callback; no early stopping

Train on 11610 samples, validate on 3870 samples
Epoch 1/10
11610/11610 [==============================] - 2s 134us/sample - loss: 1.2808 - output_loss: 1.2052 - aux_output_loss: 1.9565 - val_loss: 0.6176 - val_output_loss: 0.5763 - val_aux_output_loss: 0.9883
Epoch 2/10
11610/11610 [==============================] - 1s 94us/sample - loss: 0.5227 - output_loss: 0.4929 - aux_output_loss: 0.7902 - val_loss: 0.4817 - val_output_loss: 0.4569 - val_aux_output_loss: 0.7042
Epoch 3/10
11610/11610 [==============================] - 1s 59us/sample - loss: 0.4474 - output_loss: 0.4269 - aux_output_loss: 0.6326 - val_loss: 0.4568 - val_output_loss: 0.4371 - val_aux_output_loss: 0.6333
Epoch 4/10
11610/11610 [==============================] - 1s 60us/sample - loss: 0.4195 - output_loss: 0.4022 - aux_output_loss: 0.5741 - val_loss: 0.4333 - val_output_loss: 0.4160 - val_aux_output_loss: 0.5879
Epoch 5/10
11610/11610 [==============================] - 1s 58us/sample - loss: 0.4009 - output_loss: 0.3

In [81]:
total_loss, main_loss, aux_loss = model.evaluate([X_test_A, X_test_B], [y_test, y_test])

5160/5160 [==============================] - 0s 25us/sample - loss: 0.3646 - output_loss: 0.3555 - aux_output_loss: 0.4425


In [82]:
import tensorflow as tf
import datetime, os

In [83]:
%load_ext tensorboard.notebook

The tensorboard.notebook extension is already loaded. To reload it, use:
  %reload_ext tensorboard.notebook


In [84]:
%tensorboard --logdir=./my_logs --port=6006

Reusing TensorBoard on port 6006 (pid 78674), started -1 day, 23:40:47 ago. (Use '!kill 78674' to kill it.)

In [33]:
run_logdir2 = get_run_logdir()
run_logdir2

'./my_logs/run_2020_04_30-21_36_46'

In [22]:
total_loss  # to compare to validation result

0.3288435168968615

In [23]:
y_pred_main, y_pred_aux = model.predict([X_new_A, X_new_B])

In [24]:
y_pred_main, y_pred_aux

(array([[1.2123033],
        [1.8997341],
        [1.0523994]], dtype=float32), array([[1.322043 ],
        [2.065783 ],
        [1.1524274]], dtype=float32))

In [25]:
model.save("my_keras_modelEx13.h5")

In [ ]:
keras.optimizers.  # check all availabe optimizers.

In [ ]:
keras.losses.    # check all available loss functions.

In [ ]:
keras.metrics.  # check all available metrics.